# VacationPy
----

In [103]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [104]:
#read csv
exported_file = "Output/weather_data.csv"
weather_data_df = pd.read_csv(exported_file)
weather_data_df.head()

,City,Latitude,Longitude,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Country
0,Hobart,-42.8794,147.3294,61.68,59,75,9.22,1619827429,AU
1,Cape Town,-33.9258,18.4232,57.02,82,0,9.22,1619827326,ZA
2,São Filipe,14.8961,-24.4956,72.84,76,28,9.17,1619827437,CV
3,Butaritari,3.0707,172.7902,83.39,74,78,12.77,1619827437,KI
4,Atuona,-9.8000,-139.0333,79.27,68,0,4.43,1619827437,PF


### Humidity Heatmap

In [105]:
gmaps.configure(api_key=g_key)

In [106]:
locations = weather_data_df[['Latitude','Longitude']]
weight = weather_data_df['Humidity (%)'].astype(float)
max_humidity = weight.max()

In [107]:
#plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=weight,
                                 dissipating=False,
                                 max_intensity=max_humidity,
                                 point_radius=2)
#add heat layer
fig.add_layer(heat_layer)

#display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [108]:
#create dataframe of ideal weather conditions
ideal_weather_df = weather_data_df.loc[(weather_data_df['Temperature']>=70) & (weather_data_df['Temperature']<=80) & (weather_data_df['Wind Speed (mph)'] < 10) & (weather_data_df['Cloudiness (%)'] == 0), :]
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df['index']
ideal_weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Country
0,Atuona,-9.8000,-139.0333,79.27,68,0,4.43,1619827437,PF
1,Borazjan,29.2666,51.2159,79.25,23,0,6.49,1619827455,IR
2,Saint-Philippe,-21.3585,55.7679,70.30,78,0,6.91,1619827462,RE
3,Sakakah,29.9697,40.2064,75.20,16,0,6.91,1619827469,SA
4,Santa Cruz de la Sierra,-17.8000,-63.1667,72.01,35,0,3.74,1619827200,BO


### Hotel Map

In [109]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


hotel_list = []

for x in range(len(ideal_weather_df)):
    lat = ideal_weather_df.loc[x]['Latitude']
    lng = ideal_weather_df.loc[x]['Longitude']
    
    params = {'location':f'{lat},{lng}',
              'radius':5000,
              'types':'hotel',
              'key': g_key}
    
    response = requests.get(url, params=params).json()
    try:
        hotel_list.append(response['results'][0]['name'])
    except:
        hotel_list.append('')
print(hotel_list)

['Atuona', 'Borazjan', 'Saint-Philippe', 'Sakaka', 'Santa Cruz de la Sierra', 'Chongqing', 'Katherine', 'Tahta', 'Cayenne', 'High Rock', 'Buraydah', 'Morondava', 'Paralimni', 'Saint Pierre']


In [110]:
ideal_weather_df['Hotel Name'] = hotel_list
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City,Latitude,Longitude,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Country,Hotel Name
0,Atuona,-9.8000,-139.0333,79.27,68,0,4.43,1619827437,PF,Atuona
1,Borazjan,29.2666,51.2159,79.25,23,0,6.49,1619827455,IR,Borazjan
2,Saint-Philippe,-21.3585,55.7679,70.30,78,0,6.91,1619827462,RE,Saint-Philippe
3,Sakakah,29.9697,40.2064,75.20,16,0,6.91,1619827469,SA,Sakaka
4,Santa Cruz de la Sierra,-17.8000,-63.1667,72.01,35,0,3.74,1619827200,BO,Santa Cruz de la Sierra
5,Beibei,29.8256,106.4339,71.29,73,0,1.88,1619827478,CN,Chongqing
6,Katherine,-14.4667,132.2667,78.80,65,0,4.61,1619827497,AU,Katherine
7,Ţahţā,26.7693,31.5021,77.95,15,0,9.22,1619827508,EG,Tahta
8,Cayenne,4.9333,-52.3333,77.38,100,0,3.44,1619827523,GF,Cayenne
9,High Rock,26.6208,-78.2833,78.80,78,0,4.61,1619827531,BS,High Rock


In [111]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
location = ideal_weather_df[["Latitude", "Longitude"]]

In [112]:
# Add marker layer 
figure = gmaps.figure()

marker = gmaps.marker_layer(location, info_box_content=hotel_info)
figure.add_layer(marker)
figure.add_layer(heat_layer)
# Display figure
figure

Figure(layout=FigureLayout(height='420px'))